In [ ]:
import hyperspy.api as hys
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf

In [ ]:
ll_adr = tkf.askopenfilename()
print(ll_adr)

In [ ]:
ll = hys.load(ll_adr, signal_type="EELS")
print(ll)

In [ ]:
zlp_shift = ll.estimate_zero_loss_peak_centre().data
print(zlp_shift.shape)
spike = np.where(zlp_shift > 10)
print(spike)
for i in range(len(spike[0])):
    iy, ix = spike[0][i], spike[1][i]
    ll.data[iy, ix] = ll.data[iy-1, ix]
    print("the spectrum at %d column, %d row will be replaced by the spectrum at %d column, %d row"%(iy, ix, iy-1, ix))

In [ ]:
ll.align_zero_loss_peak(signal_range=(-0.5, 0.5))
print(ll)

In [ ]:
%matplotlib qt
ll.plot()

In [ ]:
# binning
cluster_size_y = 3
cluster_size_x = 248
stride_y = 3
stride_x = 1
ll_binned = []
print(ll.data.shape[0], ll.data.shape[1])
height = len(range(0, ll.data.shape[0]-cluster_size_y+1, stride_y))
width = len(range(0, ll.data.shape[1]-cluster_size_x+1, stride_x))
print(height, width)
for i in range(0, ll.data.shape[0]-cluster_size_y+1, stride_y):
    for j in range(0, ll.data.shape[1]-cluster_size_x+1, stride_x):
        ll_binned.append(np.sum(ll.data[i:i+cluster_size_y, j:j+cluster_size_x], axis=(0, 1)))
        
ll_binned = np.asarray(ll_binned).reshape(height, width, -1)
ll_binned = hys.signals.Signal1D(ll_binned)
ll_binned.metadata = ll.metadata
ll_binned.axes_manager = ll.axes_manager
ll_binned.axes_manager[0].size = width
ll_binned.axes_manager[1].size = height
ll_binned.axes_manager[0].scale = ll.axes_manager[0].scale * cluster_size_x
ll_binned.axes_manager[1].scale = ll.axes_manager[1].scale * cluster_size_y
print(ll_binned.axes_manager)

In [ ]:
th = ll.estimate_elastic_scattering_threshold(start=0.1)
print(th.data)

In [ ]:
ll_thickness = ll.estimate_thickness(threshold=168)
print(ll_thickness.data)

In [ ]:
%matplotlib qt
ll_thickness.plot()

In [ ]:
%matplotlib qt
ll.remove_background(zero_fill=True, fast=False)

In [ ]:
%matplotlib qt
ll.plot()

In [ ]:
ll = ll.isig[0.0:18.0]
ll_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(ll.data, 0, 3), 0, 3))
print(ll_dm)

ll.save(ll_adr[:-4]+"_alined_cropped.hdf5")
ll_dm.save(ll_adr[:-4]+"_alined_dm_import_cropped.hdf5")

In [ ]:
ll_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(ll.data, 0, 3), 0, 3))
print(ll_dm)

ll.save(ll_adr[:-4]+"_alined.hdf5")
ll_dm.save(ll_adr[:-4]+"_alined_dm_import.hdf5")

In [ ]:
ll_sum = np.sum(ll.data, axis=2)
print(ll_sum.shape)
print(np.max(ll_sum), np.min(ll_sum), np.mean(ll_sum))

In [ ]:
ll_nor = ll.deepcopy()
ll_nor.data = ll_nor.data / ll_sum[:, :, np.newaxis]
print(np.unique(np.sum(ll_nor, axis=2)))

In [ ]:
ll_nor.save(ll_adr[:-4]+"_total_intensity_normalized.hdf5")

In [ ]:
ll_cropped = ll.isig[3.0:18.0]
ll_sum = np.sum(ll_cropped.data, axis=2)
print(ll_sum.shape)
print(np.max(ll_sum), np.min(ll_sum), np.mean(ll_sum))

In [ ]:
ll_cropped_nor = ll_cropped.deepcopy()
ll_cropped_nor.data = ll_cropped_nor.data / ll_sum[:, :, np.newaxis]
print(np.unique(np.sum(ll_cropped_nor, axis=2)))
ll_cropped_nor.save(ll_adr[:-4]+"_total_intensity_normalized.hdf5")